https://stepik.org/lesson/1576200/step/7

https://github.com/selfedu-rus/neuro-pytorch/blob/main/solves/4.3.5

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim


# сюда копируйте класс CharsDataset из предыдущего подвига
class CharsDataset(data.Dataset):
    def __init__(self, prev_chars=10):
        super().__init__()
        self.prev_chars = prev_chars
        self.rows = _global_var_text
        all_text = "".join(self.rows).lower()

        self.alphabet = set(all_text.lower()) # сэт из символов текста
        self.int_to_alpha = dict(enumerate(sorted(self.alphabet))) # отсортированный cловарь
        self.alpha_to_int = {b: a for a, b in self.int_to_alpha.items()} # обратный словарь

        # Генерация данных
        self.data = []
        for row in self.rows:
            row = row.lower()
            for i in range(len(row) - prev_chars):
                self.data.append(([self.alpha_to_int[c] for c in row[i:i+prev_chars]],
                                  self.alpha_to_int[row[i+prev_chars]]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        # one-hot-конвертация:
        x_onehot = torch.zeros(self.prev_chars, len(self.alphabet))
        x_onehot[torch.arange(self.prev_chars), torch.tensor(x)] = 1

        return x_onehot, torch.tensor(y)

# здесь объявляйте класс модели нейронной сети
class RNN_Model(nn.Module):
    def __init__(self, in_features, out_linear):
        super().__init__()
        self.rnn = nn.RNN(in_features, 32, batch_first=True)
        self.linear = nn.Linear(32, out_linear, bias=True)

    def forward(self, x):
        x, h = self.rnn(x)
        return self.linear(h)

# # Переменная для теста
# _global_var_text = [
# "Как я отмечал во введении, простейшая НС – персептрон, представляет собой",
# "Это классический пример полносвязной сети",
# "Каждая связь между нейронами имеет определенный"
# ]

# сюда копируйте объекты d_train и train_data
d_train = CharsDataset(prev_chars=10)
train_data = data.DataLoader(d_train, batch_size=8, shuffle=True)

n = len(d_train.alphabet) # число входов и выходовв сети (размер словаря)

model = RNN_Model(n, n)

# Оптимизатор и функция потерь
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_func = nn.CrossEntropyLoss()

epochs = 1 # число эпох (это конечно, очень мало, в реальности нужно от 100 и более)
model.train() # переводим модель в режим обучения

# Цикл обучепния
for _e in range(epochs):
    for x_train, y_train in train_data:
        predict = model(x_train).squeeze(0)
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

model.eval() # переводим модель в режим предсказания
predict = "нейронная сеть ".lower() # начальная фраза
total = 20 # число прогнозируемых символов (дополнительно к начальной фразе)

# выполните прогноз следующих total символов

# Цикл прогноза
for _ in range(total):
    # Подготовка для one-hot-конвертации:
    x_input = torch.zeros(1, 10, n)

    # Делаем срез последних 10-ти символов
    for i, key in enumerate(predict[-10:]):
        # Окончание one-hot-конвертации
        # Помечаем единицей индекс, соответствующий символу (по ключу в словаре)
        x_input[0, i, d_train.alpha_to_int[key]] = 1

    # Прогноз модели по сформированному срезу
    output = model(x_input)

    # Выбираем символ с максимальной вероятностью (его ключ/индекс)
    key = torch.argmax(output).item()

    # По индексу выбираем из словаря предсказанный символ и приклеиваем к строке
    predict += d_train.int_to_alpha[key]

# выведите полученную строку на экран
print(predict)
len(predict)

нейронная сеть                     


35

In [ ]:
# @title тестовое решение для проверки системы
# https://github.com/selfedu-rus/neuro-pytorch/blob/main/solves/4.3.5
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim


class CharsDataset(data.Dataset):
    def __init__(self, prev_chars=7):
        self.prev_chars = prev_chars

        self.lines = _global_var_text
        self.alphabet = set(("".join(self.lines)).lower())
        self.int_to_alpha = dict(enumerate(sorted(self.alphabet)))
        self.alpha_to_int = {b: a for a, b in self.int_to_alpha.items()}
        self.num_characters = len(self.alphabet)
        self.onehots = torch.eye(self.num_characters)

        data = []
        targets = []

        for i, t in enumerate(self.lines):
            t = t.lower()
            for item in range(len(t)-self.prev_chars):
                data.append([self.alpha_to_int[t[x]] for x in range(item, item + self.prev_chars)])
                targets.append(self.alpha_to_int[t[item+self.prev_chars]])

        self.data = torch.tensor(data)
        self.targets = torch.tensor(targets)

        self.length = len(data)

    def __getitem__(self, item):
        return self.onehots[self.data[item]], self.targets[item]

    def __len__(self):
        return self.length


class TextRNN(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.hidden_size = 32
        self.in_features = in_features
        self.out_features = out_features

        self.rnn = nn.RNN(in_features, self.hidden_size, batch_first=True)
        self.out = nn.Linear(self.hidden_size, out_features)

    def forward(self, x):
        x, h = self.rnn(x)
        y = self.out(h)
        return y


# сюда копируйте объекты d_train и train_data
d_train = CharsDataset(prev_chars=10)
train_data = data.DataLoader(d_train, batch_size=8, shuffle=True)

model = TextRNN(d_train.num_characters, d_train.num_characters)

optimizer = optim.Adam(params=model.parameters(), lr=0.01)
loss_func = nn.CrossEntropyLoss()

epochs = 1 # число эпох
model.train()

for _e in range(epochs):
    for x_train, y_train in train_data:
        predict = model(x_train).squeeze(0)
        loss = loss_func(predict, y_train.long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

model.eval()
predict = "нейронная сеть ".lower() # начальная фраза
total = 20 # число прогнозируемых символов (дополнительно к начальной фразе)

for _ in range(total):
    _data = d_train.onehots[[d_train.alpha_to_int[predict[-x]] for x in range(d_train.prev_chars, 0, -1)]]
    with torch.no_grad():
        p = model(_data.unsqueeze(0)).squeeze(0)
    indx = torch.argmax(p, dim=1)
    predict += d_train.int_to_alpha[indx.item()]

print(predict)
len(predict)

нейронная сеть ееееееееееееееееееее


35